In [ ]:

import pandas as pd
from scipy.stats import zscore

# Escolha a opção de normalização: 'zscore', 'minmax' ou 'none'
NORMALIZE_OPTION = 'none'

# Carrega os dados clínicos dos pacientes
df_clinical_patient = pd.read_csv('_data_clinical_patient.txt', sep='\t', header=0, index_col=0)
# mostra as 4 últimas linhas
df_clinical_patient = df_clinical_patient.tail(-4)
df_clinical_patient.columns
# Carrega os dados de CNA (Copy Number Alteration) e remove a coluna 'Entrez_Gene_Id'
df_cna_mutations = pd.read_csv('_data_cna.txt', sep='\t', header=0, index_col=0)
df_cna_mutations = df_cna_mutations.drop(columns=['Entrez_Gene_Id'])

# Carrega os dados de expressão de mRNA e remove a coluna 'Entrez_Gene_Id'
df_mrna = pd.read_csv('_data_mrna_agilent_microarray_zscores_ref_all_samples.txt', sep='\t', header=0, index_col=0)
df_mrna = df_mrna.drop(columns=['Entrez_Gene_Id'])

# Função para remover valores ausentes (NaN) e imprimir informação sobre a remoção
def remove_missing_values(data, data_name):
    before = data.shape
    data.dropna(inplace=True)
    after = data.shape
    removed_rows = before[0] - after[0]
    if removed_rows > 0:
        print(f"{removed_rows} linhas com valores ausentes foram removidas de {data_name}.")
    return data

# Aplica a função de remoção de NaN aos datasets
df_clinical_patient = remove_missing_values(df_clinical_patient, "df_clinical_patient")
df_cna_mutations = remove_missing_values(df_cna_mutations, "df_cna_mutations")
df_mrna = remove_missing_values(df_mrna, "df_mrna")

def normalize_data(data, method):
    if method == 'zscore':
        return data.apply(zscore)
    elif method == 'minmax':
        return (data - data.min()) / (data.max() - data.min())
    else:
        return data

df_cna_mutations = normalize_data(df_cna_mutations, NORMALIZE_OPTION)
df_mrna = normalize_data(df_mrna, NORMALIZE_OPTION)

print(df_mrna)

# Transpõe as tabelas de mRNA e CNA para que os genes sejam colunas e os pacientes sejam linhas
df_mrna = df_mrna.transpose()
df_cna_mutations = df_cna_mutations.transpose()
df_cna_mutations

# Adiciona sufixos para distinguir as colunas de mRNA e CNA ao juntar os dados
df_mrna.columns = df_mrna.columns.map(lambda x: str(x) + '_MRNA')
df_cna_mutations.columns = df_cna_mutations.columns.map(lambda x: str(x) + '_CNA')

# Une os datasets de mRNA e CNA com base no índice (ID do paciente)
df_mrna_and_cna_mutations = pd.merge(df_mrna, df_cna_mutations, left_index=True, right_index=True)

# Une o dataset clínico ao conjunto combinado de mRNA e CNA
df_final = pd.merge(df_clinical_patient, df_mrna_and_cna_mutations, left_index=True, right_index=True)
df_final

# Imprime as dimensões de cada dataframe para verificar o tamanho e consistência dos dados
print(df_mrna.shape)
print(df_cna_mutations.shape)
print(df_mrna_and_cna_mutations.shape)
print(df_clinical_patient.shape)
print(df_final.shape)

# Salva o dataframe final pré-processado num ficheiro CSV ou pkl
df_final.to_csv('dataset_preprocessed_test.csv')
#df_final.to_pickle("'dataset_preprocessed_test.pkl")
